In [1]:
region_name = "TDS_ELAISN1"  #_ELAISN1
redo_src_sel = True  # Redo source selection in dedicated cell below
field_id = "GNU2597523790556561408"
art_nr = 0
dd_arts = {0:{"field_id":"GNU2597523790556561408", "ra":241.288, "dec":55.352, "rad":0.55},  # All field
           1:{"field_id":"GNU2597523790556561408", "ra":241.58, "dec":55.43, "rad":0.01},    # Line artifact
          } 
dd_art = dd_arts[art_nr]

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
import numpy as np

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, sel_sources, dd_filter2id

#Load region and field
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)
display(rg.tt_fields)
fd = rg.get_field(field_id=dd_art["field_id"], load_method="FITS", add_field=False)

field_id,field_name,ra,dec,observatory,obs_filter,fov_diam,sel,nr_vis,time_bin_size_sum,time_start,time_stop,rg_fd_id
,,deg,deg,,,deg,,,s,d,d,
bytes32,bytes32,float64,float64,bytes22,bytes8,float32,bool,int32,float32,float64,float64,int64
GNU2597312684324028416,PS_ELAISN1_MOS10,242.50638203682297,55.9801443588634,GALEX,NUV,1.2,True,24,29566.4,54960.17787037037,56821.33210648148,1
GNU2597383053068206080,PS_ELAISN1_MOS11,244.57200462478102,55.1825511621234,GALEX,NUV,1.2,True,25,31369.5,54960.24638888889,56815.50054271557,2
GNU2597453421812383744,PS_ELAISN1_MOS12,242.896209389035,55.0003489374844,GALEX,NUV,1.2,True,21,26349.451,54960.314884259256,56814.569061234084,3
GNU2597523790556561408,PS_ELAISN1_MOS13,241.28881797574297,55.3529870930308,GALEX,NUV,1.2,True,22,27810.902,54960.38337962963,56819.53759259259,4
GNU2597594159300739072,PS_ELAISN1_MOS14,243.95513153083402,54.1978605194106,GALEX,NUV,1.2,True,22,26881.852,54960.451886574076,56810.60611111111,5
GNU2597664528044916736,PS_ELAISN1_MOS15,242.396514585217,53.9999872930018,GALEX,NUV,1.2,True,22,28044.65,54960.52040509259,56810.67461805556,6
GNU2597734896789094400,PS_ELAISN1_MOS16,241.37385712300306,54.4519391954855,GALEX,NUV,1.2,True,21,26780.799,54960.588912037034,56807.7931854022,7
GFU2597312684324028416,PS_ELAISN1_MOS10,242.50638203682297,55.9801443588634,GALEX,FUV,1.2,True,5,6920.1,54960.17787037037,56504.46356481482,8


In [3]:
from regions import CircleSkyRegion
from astropy.coordinates import SkyCoord
from astropy import units as uu

fig_art = plt.figure("Sky map artifact",figsize=(8, 7))

sky_center = SkyCoord(dd_art["ra"]*uu.deg, dd_art["dec"]*uu.deg, frame="icrs")
sky_art= CircleSkyRegion(center=sky_center, radius=dd_art["rad"]*uu.deg)
plt.cla()
ax, wcs_art = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = sky_art, fig=fig_art)

#fig, ax = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = fd.get_sky_region())

print("ellipse:blue , class_star:green, artifact:yellow")
lw= 0.25
sel_s2n = fd.tt_detections["s2n"]>3.5
fd.tt_detections["sel"]  = sel_s2n
tt_det = fd.tt_detections
kdet_s2n = {"marker":"o","color":"r","markersize": 5.5,"alpha":1,"markeredgewidth": lw}
ax, tt_det = vvis.plot_sky_sources( fd.tt_detections[sel_s2n], src_id=None,src_kwargs=kdet_s2n, sky_region_wcs = (sky_art,wcs_art), only_selected=True)

if dd_art["rad"]<0.49:
    sel_ell = tt_det["ellip_world"]>0.5
    tt_det["sel"]  = sel_ell
    kdet_ell = {"marker":"o","color":"b","markersize": 4.5,"alpha":1,"markeredgewidth": lw}
    ax, tt_det_ell = vvis.plot_sky_sources(tt_det, src_id=None,src_kwargs=kdet_ell, sky_region_wcs = (sky_art,wcs_art), only_selected=True)
    
    sel_cs = tt_det["class_star"]<0.6
    tt_det["sel"]  = sel_cs
    kdet_cs = {"marker":"o","color":"g","markersize": 2.5,"alpha":1,"markeredgewidth": lw}
    ax, tt_det_cs = vvis.plot_sky_sources(tt_det, src_id=None,src_kwargs=kdet_cs, sky_region_wcs = (sky_art,wcs_art), only_selected=True)
    
    sel_art = tt_det["artifacts"]>0
    tt_det["sel"]  = sel_art
    kdet_art = {"marker":"o","color":"y","markersize": 3.5,"alpha":1,"markeredgewidth": lw}
    ax, tt_det_art = vvis.plot_sky_sources(tt_det, src_id=None,src_kwargs=kdet_art, sky_region_wcs = (sky_art,wcs_art), only_selected=True)

plt.show()

fig_dist, axs = plt.subplots(1,3,figsize=(9, 4),squeeze=False)
axs = axs.flatten()
axs[0].hist(tt_det["ellip_world"])
axs[0].set_xlabel("ellip_world")
axs[1].hist(tt_det["class_star"])
axs[1].set_xlabel("class_star")
axs[2].hist(tt_det["artifacts"])
axs[2].set_xlabel("artifacts")
plt.show()
display(tt_det_ell,tt_det_cs,tt_det_art)

SyntaxError: expected ':' (3924714013.py, line 22)

Interresting cases (ICCS):
* 241.49, 55.50: Extended sourse cut out by class_star cut, but not ellipse

In [ ]:
#selections = { "table":"tt_detections","presel_type":"and", "sel_type":"and",
#              "range": {"s2n":[3.5,np.inf],"r_fov":[0.0,0.55], "class_star": [0.6,1.0] ,"chkobj_type": [-0.5,0.5]}}
#print(selections)
#fd.select_rows(selections, remove_unselected=False)